<a href="https://colab.research.google.com/github/Plananas/AI-and-Data-Mining/blob/main/AI_and_Datamining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Optimizing Patient Flow: Predicting and Reducing Wait Times in Healthcare Appointments**



**Objective:**
Reduce patient wait times by analyzing historical data and identifying factors contributing to delays.
Develop a predictive model to estimate wait times based on key features.

Link to the data source: https://www.kaggle.com/datasets/carogonzalezgaltier/medical-appointment-scheduling-system

This program will be using the supplied data to reduce the wait times of patients.

Make sure this has been removed and changed before it is marked

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd

appointments_file_path = "/content/drive/My Drive/Data/appointments.csv"
patients_file_path = "/content/drive/My Drive/Data/patients.csv"
slots_file_path = "/content/drive/My Drive/Data/slots.csv"
appointments = pd.read_csv(appointments_file_path)
patients = pd.read_csv(patients_file_path)
slots = pd.read_csv(slots_file_path)

In [4]:
appointments.head()

,appointment_id,slot_id,scheduling_date,appointment_date,appointment_time,scheduling_interval,status,check_in_time,appointment_duration,start_time,end_time,waiting_time,patient_id,sex,age,age_group
0,138,1,2014-12-28,2015-01-01,08:00:00,4,did not attend,NaN,NaN,NaN,NaN,NaN,8285,Male,37,35-39
1,146,23,2014-12-29,2015-01-01,13:30:00,3,did not attend,NaN,NaN,NaN,NaN,NaN,5972,Male,84,80-84
2,21,24,2014-12-17,2015-01-01,13:45:00,15,attended,13:36:45,5.2,13:37:57,13:43:09,1.2,6472,Male,77,75-79
3,233,25,2014-12-31,2015-01-01,14:00:00,1,attended,13:59:32,28.9,14:00:40,14:29:34,1.1,5376,Female,37,35-39
4,90,26,2014-12-26,2015-01-01,14:15:00,6,cancelled,NaN,NaN,NaN,NaN,NaN,8028,Male,72,70-74


In [5]:
appointments.describe()

,appointment_id,slot_id,scheduling_interval,appointment_duration,waiting_time,patient_id,age
count,111488.000000,111488.000000,111488.000000,86032.000000,86032.000000,111488.000000,111488.000000
mean,55744.500000,51854.320438,7.190146,17.479525,44.086509,13783.228581,57.205206
std,32183.957743,29945.828944,6.146793,11.064225,40.788791,9722.943016,20.158119
min,1.000000,1.000000,1.000000,0.000000,0.600000,1.000000,15.000000
25%,27872.750000,25909.750000,2.000000,8.600000,12.600000,5702.000000,40.000000
50%,55744.500000,51852.000000,5.000000,15.800000,33.500000,11816.000000,59.000000
75%,83616.250000,77796.250000,10.000000,24.700000,64.600000,20736.250000,74.000000
max,111488.000000,104159.000000,30.000000,58.700000,297.300000,36697.000000,100.000000


In [7]:
appointments.info()
slots.info()
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111488 entries, 0 to 111487
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   appointment_id        111488 non-null  int64  
 1   slot_id               111488 non-null  int64  
 2   scheduling_date       111488 non-null  object 
 3   appointment_date      111488 non-null  object 
 4   appointment_time      111488 non-null  object 
 5   scheduling_interval   111488 non-null  int64  
 6   status                111488 non-null  object 
 7   check_in_time         86032 non-null   object 
 8   appointment_duration  86032 non-null   float64
 9   start_time            86032 non-null   object 
 10  end_time              86032 non-null   object 
 11  waiting_time          86032 non-null   float64
 12  patient_id            111488 non-null  int64  
 13  sex                   111488 non-null  object 
 14  age                   111488 non-null  int64  
 15  